In [1]:
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt

data_path='data'

In [2]:
target_pollutants='SO2 CO PM10 PM25 O3'.split()

## Madrid

In [3]:
magnitude_names={1:'SO2', 6:'CO',7:'NO',8:'NO2', 9:'PM25', 10:'PM10', 12:'NOX', 14:'O3',
                20:'TOL', 30:'BEN', 35:'EBE', 37:'MXY', 38:'PXY', 39:'OXY', 42:'TCH', 43:'CH4'}

In [4]:
def read_ap_madrid_file(filename):
    madrid_apdata_df= pd.read_csv(os.path.join(data_path, 'madrid_airpollution',filename ),sep=';', 
                                  dtype={'PROVINCIA':str, 'MUNICIPIO':str, 'ESTACION':str,
                                        'ANO':str, 'MES':str, 'DIA':str})
    madrid_apdata_df['id']=madrid_apdata_df.apply(lambda r: r['PROVINCIA']+r['MUNICIPIO'].zfill(3)+r['ESTACION'].zfill(3), axis=1)
    
    target_ap_madrid='28079008 28079049 28079004 28079047'.split()
    madrid_apdata_df= madrid_apdata_df[madrid_apdata_df['id'].isin(target_ap_madrid)]
    
    madrid_apdata_df['date']=madrid_apdata_df.apply(lambda r: r['ANO']+'-'+r['MES']+'-'+r['DIA'], axis=1)
    madrid_apdata_df['date']= pd.to_datetime(madrid_apdata_df['date'])
    madrid_apdata_df['pollutant']= madrid_apdata_df['MAGNITUD'].apply(lambda v: magnitude_names[v])
    
    target_columns=['H'+str(v).zfill(2) for v in range(0,25)]
    target_columns.append('id')
    target_columns.append('date')
    target_columns.append('pollutant')
    
    madrid_apdata_df= madrid_apdata_df.drop(columns=madrid_apdata_df.columns.difference(target_columns))
    #madrid_apdata_df.head()
    return madrid_apdata_df

In [5]:
md_initdate= '2023/09/13'
md_enddate= '2023/10/03'

oct_md_data= read_ap_madrid_file('oct_mo23.csv')
sep_md_data= read_ap_madrid_file('sep_mo23.csv')
md_ap_df = pd.concat([sep_md_data,oct_md_data],axis=0)

md_ap_df= md_ap_df[(md_ap_df['date']>= md_initdate) & (md_ap_df['date']<= md_enddate)]
md_ap_df= md_ap_df[md_ap_df['pollutant'].isin(target_pollutants)]
md_ap_df= md_ap_df.reset_index(drop=True)
md_ap_df.columns= ['H'+str(int(c[1:])-1).zfill(2) if c not in 'id date pollutant'.split() else c for c in md_ap_df.columns]
md_ap_df.head()

,H00,H01,H02,H03,H04,H05,H06,H07,H08,H09,...,H17,H18,H19,H20,H21,H22,H23,id,date,pollutant
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,28079004,2023-09-13,SO2
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,28079004,2023-09-14,SO2
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,28079004,2023-09-15,SO2
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,28079004,2023-09-16,SO2
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,28079004,2023-09-17,SO2


In [6]:
md_ap_df.tail()

,H00,H01,H02,H03,H04,H05,H06,H07,H08,H09,...,H17,H18,H19,H20,H21,H22,H23,id,date,pollutant
205,28.0,23.0,22.0,21.0,19.0,19.0,25.0,16.0,14.0,34.0,...,27.0,22.0,25.0,30.0,37.0,56.0,41.0,28079047,2023-10-02,PM10
206,29.0,26.0,20.0,22.0,16.0,23.0,14.0,23.0,28.0,32.0,...,30.0,19.0,23.0,28.0,36.0,44.0,48.0,28079047,2023-10-03,PM10
207,38.0,28.0,21.0,27.0,36.0,47.0,51.0,45.0,55.0,54.0,...,106.0,87.0,77.0,74.0,40.0,19.0,8.0,28079049,2023-10-01,O3
208,16.0,32.0,30.0,29.0,38.0,47.0,45.0,31.0,14.0,8.0,...,96.0,87.0,78.0,60.0,22.0,9.0,8.0,28079049,2023-10-02,O3
209,5.0,11.0,38.0,46.0,43.0,39.0,34.0,27.0,14.0,13.0,...,98.0,89.0,88.0,82.0,61.0,54.0,26.0,28079049,2023-10-03,O3


In [7]:
def get_pollutants_per_sensor(ad_df):
    grouped_df = ad_df.groupby(['id', 'date'])
    pollutants_per_sensor_lst=[]
    for (id_, date_), group_df in grouped_df:
        print(f"({id_}, {date_}): {group_df['pollutant'].tolist()}")
        pollutants_per_sensor_lst.append((id_,date_,' '.join(group_df['pollutant'].tolist())))

    pollutants_per_sensor_df= pd.DataFrame(pollutants_per_sensor_lst)
    # We can see that the sensors resport the same set of pollutants during all the study period
    print(pollutants_per_sensor_df.groupby(0).nunique(2))

    pollutants_per_sensor_df= pollutants_per_sensor_df.drop(columns=[1])
    pollutants_per_sensor_df= pollutants_per_sensor_df.drop_duplicates(0,keep='first')
    pollutants_per_sensor_df= pollutants_per_sensor_df.reset_index(drop=True)
    pollutants_per_sensor_df.columns='id pollutants'.split()
    return pollutants_per_sensor_df

In [8]:
pollutants_per_sensor_df= get_pollutants_per_sensor(md_ap_df)
pollutants_per_sensor_df.head()

(28079004, 2023-09-13 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-14 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-15 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-16 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-17 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-18 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-19 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-20 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-21 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-22 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-23 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-24 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-25 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-26 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-27 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-28 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-29 00:00:00): ['SO2', 'CO']
(28079004, 2023-09-30 00:00:00): ['SO2', 'CO']
(28079004, 2023-10-01 00:00:00): ['SO2', 'CO']
(28079004, 2023-10-02 00:00:00): ['SO2', 'CO']
(28079004, 2023-10-03 00:00:00): ['SO2', 'CO']
(28079008, 20

,id,pollutants
0,28079004,SO2 CO
1,28079008,SO2 CO PM25 PM10 O3
2,28079047,PM25 PM10
3,28079049,O3


In [9]:
md_ap_df.to_csv(os.path.join(data_path, '02_madrid_airpollution_clean.csv'))
pollutants_per_sensor_df.to_csv(os.path.join(data_path, '02_madrid_pollutants_per_sensor.csv'))

## Bilbao

Diccionario para homogeneizar los nombres de los polutantes

In [10]:
bb_magnitude_dict={
    'Benceno (µg/m3)': 'BEN',
    'CO (mg/m3)': 'CO',
    'CO 8h (mg/m3)': 'CO-8h',
    'Etilbenceno (µg/m3)': 'EBE',
    'NO (µg/m3)': 'NO',
    'NO2 (µg/m3)': 'NO2',
    'NOX (µg/m3)': 'NOX',
    'Ortoxileno (µg/m3)': 'OXY',
    'PM10 (µg/m3)': 'PM10',
    'PM2,5 (µg/m3)': 'PM25',
    'SO2 (µg/m3)': 'SO2',
    'Tolueno (µg/m3)': 'TOL',
    'O3 (µg/m3)': 'O3',
    'O3 8h (µg/m3)': 'O3-8h'
}

In [11]:
bb_initdate= '2023/09/18'
bb_enddate= '2023/10/03'

In [12]:
target_ap_bilbao=['MAZARREDO.csv', 'M_DIAZ_HARO.csv']
bb_airpollution_files = [archivo for archivo in os.listdir(os.path.join(data_path, 'bilbao_airpollution')) if not ((archivo.startswith('.'))) and (archivo in target_ap_bilbao)]
#a=bb_airpollution_files[0]

bb_ap_dflist=[]
for a in bb_airpollution_files:

    df= pd.read_csv(os.path.join(data_path, 'bilbao_airpollution', a),encoding='unicode_escape', sep=';', decimal=',')
    df['Date']= pd.to_datetime(df['Date'], dayfirst=True)
    df= df.sort_values(['Date', 'Hour  (GMT)']).reset_index(drop=True)

    df= df.interpolate(method='linear', limit_direction='both')
    df['Hour  (GMT)']= df['Hour  (GMT)'].apply(lambda x: 'H'+str(int(x.split(':')[0])-1).zfill(2))

    df_pivoted= df.pivot(index="Date", columns='Hour  (GMT)')
    df_pivoted= df_pivoted.stack(level=0).reset_index()

    df_pivoted['level_1']= df_pivoted['level_1'].apply(lambda x: bb_magnitude_dict[x])
    
    df_pivoted['id']= a.rsplit('.',1)[0]
    df_pivoted= df_pivoted.rename(columns={'Date': 'date', 'level_1':'pollutant'})
    df_pivoted= df_pivoted.reset_index(drop=True)
    df_pivoted= df_pivoted[(df_pivoted['date']>= bb_initdate) & (df_pivoted['date']<=bb_enddate)]
    
    bb_ap_dflist.append(df_pivoted)

bb_ap_df= pd.concat(bb_ap_dflist)
bb_ap_df= bb_ap_df[bb_ap_df['pollutant'].isin(target_pollutants)]
bb_ap_df= bb_ap_df.reset_index(drop=True)
bb_ap_df.head()

C:\Users\ferna\AppData\Local\Temp\ipykernel_13948\1929267969.py:12: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df= df.interpolate(method='linear', limit_direction='both')
C:\Users\ferna\AppData\Local\Temp\ipykernel_13948\1929267969.py:16: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_pivoted= df_pivoted.stack(level=0).reset_index()
C:\Users\ferna\AppData\Local\Temp\ipykernel_13948\1929267969.py:12: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df= df.interpolate(method='linear', limit_direction='both')
C:\Users\ferna\AppData

Hour (GMT),date,pollutant,H00,H01,H02,H03,H04,H05,H06,H07,...,H15,H16,H17,H18,H19,H20,H21,H22,H23,id
0,2023-09-18,CO,0.20,0.20,0.22,0.22,0.22,0.23,0.23,0.23,...,0.21,0.20,0.23,0.24,0.27,0.27,0.27,0.25,0.25,MAZARREDO
1,2023-09-18,PM10,8.00,4.00,3.00,3.00,4.00,5.00,6.00,8.00,...,11.00,11.00,12.00,12.00,12.00,11.00,9.00,8.00,7.00,MAZARREDO
2,2023-09-18,PM25,3.00,2.00,1.00,1.00,1.00,2.00,2.00,2.00,...,5.00,5.00,6.00,6.00,6.00,5.00,4.00,4.00,3.00,MAZARREDO
3,2023-09-18,SO2,9.00,7.00,8.00,7.00,7.00,7.00,7.00,7.00,...,7.00,7.00,6.00,7.00,7.00,7.00,7.00,7.00,7.00,MAZARREDO
4,2023-09-19,CO,0.26,0.25,0.26,0.30,0.31,0.33,0.38,0.36,...,0.26,0.23,0.23,0.26,0.30,0.32,0.32,0.31,0.39,MAZARREDO


In [13]:
bb_ap_df.tail()

Hour (GMT),date,pollutant,H00,H01,H02,H03,H04,H05,H06,H07,...,H15,H16,H17,H18,H19,H20,H21,H22,H23,id
139,2023-10-03,CO,0.3,0.27,0.3,0.33,0.33,0.36,0.42,0.45,...,0.26,0.28,0.3,0.3,0.32,0.29,0.3,0.26,0.23,M_DIAZ_HARO
140,2023-10-03,O3,37.0,36.00,30.0,17.00,13.00,10.00,10.00,11.00,...,33.00,30.00,32.0,39.0,44.00,50.00,46.0,48.00,57.00,M_DIAZ_HARO
141,2023-10-03,PM10,20.0,18.00,20.0,24.00,24.00,24.00,27.00,32.00,...,14.00,8.00,8.0,6.0,7.00,12.00,20.0,15.00,14.00,M_DIAZ_HARO
142,2023-10-03,PM25,15.0,14.00,15.0,19.00,19.00,18.00,19.00,20.00,...,7.00,5.00,5.0,4.0,5.00,7.00,9.0,9.00,6.00,M_DIAZ_HARO
143,2023-10-03,SO2,5.0,5.00,5.0,5.00,4.00,5.00,5.00,5.00,...,5.00,4.00,5.0,5.0,4.00,4.00,4.0,4.00,4.00,M_DIAZ_HARO


In [14]:
bb_pollutants_per_sensor_df= get_pollutants_per_sensor(bb_ap_df)
bb_pollutants_per_sensor_df.head()

(MAZARREDO, 2023-09-18 00:00:00): ['CO', 'PM10', 'PM25', 'SO2']
(MAZARREDO, 2023-09-19 00:00:00): ['CO', 'PM10', 'PM25', 'SO2']
(MAZARREDO, 2023-09-20 00:00:00): ['CO', 'PM10', 'PM25', 'SO2']
(MAZARREDO, 2023-09-21 00:00:00): ['CO', 'PM10', 'PM25', 'SO2']
(MAZARREDO, 2023-09-22 00:00:00): ['CO', 'PM10', 'PM25', 'SO2']
(MAZARREDO, 2023-09-23 00:00:00): ['CO', 'PM10', 'PM25', 'SO2']
(MAZARREDO, 2023-09-24 00:00:00): ['CO', 'PM10', 'PM25', 'SO2']
(MAZARREDO, 2023-09-25 00:00:00): ['CO', 'PM10', 'PM25', 'SO2']
(MAZARREDO, 2023-09-26 00:00:00): ['CO', 'PM10', 'PM25', 'SO2']
(MAZARREDO, 2023-09-27 00:00:00): ['CO', 'PM10', 'PM25', 'SO2']
(MAZARREDO, 2023-09-28 00:00:00): ['CO', 'PM10', 'PM25', 'SO2']
(MAZARREDO, 2023-09-29 00:00:00): ['CO', 'PM10', 'PM25', 'SO2']
(MAZARREDO, 2023-09-30 00:00:00): ['CO', 'PM10', 'PM25', 'SO2']
(MAZARREDO, 2023-10-01 00:00:00): ['CO', 'PM10', 'PM25', 'SO2']
(MAZARREDO, 2023-10-02 00:00:00): ['CO', 'PM10', 'PM25', 'SO2']
(MAZARREDO, 2023-10-03 00:00:00): ['CO',

,id,pollutants
0,MAZARREDO,CO PM10 PM25 SO2
1,M_DIAZ_HARO,CO O3 PM10 PM25 SO2


In [15]:
bb_ap_df.to_csv(os.path.join(data_path, '02_bilbao_airpollution_clean.csv'))
bb_pollutants_per_sensor_df.to_csv(os.path.join(data_path, '02_bilbao_pollutants_per_sensor.csv'))

In [16]:
print("That's all folks!")

That's all folks!


## Test code (do not run)

In [17]:
df= pd.read_csv(os.path.join(data_path, 'bilbao_airpollution', bb_airpollution_files[0]),encoding='unicode_escape', sep=';', decimal=',')
df['Date']= pd.to_datetime(df['Date'], dayfirst=True)
df= df.sort_values(['Date', 'Hour  (GMT)']).reset_index(drop=True)
df

,Date,Hour (GMT),Benceno (µg/m3),CO (mg/m3),CO 8h (mg/m3),Etilbenceno (µg/m3),NO (µg/m3),NO2 (µg/m3),NOX (µg/m3),Ortoxileno (µg/m3),PM10 (µg/m3),"PM2,5 (µg/m3)",SO2 (µg/m3),Tolueno (µg/m3)
0,2023-01-01,01:00,NaN,0.19,0.17,NaN,3.0,12.0,16.0,NaN,33.0,12.0,5.0,NaN
1,2023-01-01,02:00,NaN,0.19,0.17,NaN,2.0,8.0,12.0,NaN,31.0,12.0,4.0,NaN
2,2023-01-01,03:00,0.09,0.21,0.18,0.04,1.0,7.0,9.0,0.40,31.0,12.0,4.0,0.77
3,2023-01-01,04:00,0.09,0.22,0.19,0.06,1.0,12.0,14.0,0.44,33.0,13.0,4.0,0.79
4,2023-01-01,05:00,0.06,0.23,0.19,0.00,2.0,20.0,22.0,0.41,31.0,13.0,4.0,0.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2023-12-31,20:00,0.31,0.31,0.22,0.00,9.0,57.0,70.0,0.48,25.0,19.0,7.0,0.86
8756,2023-12-31,21:00,0.52,0.32,0.24,0.00,9.0,57.0,69.0,0.58,33.0,28.0,6.0,1.07
8757,2023-12-31,22:00,0.51,0.32,0.26,0.05,4.0,39.0,45.0,0.62,16.0,12.0,4.0,1.19
8758,2023-12-31,23:00,NaN,0.31,0.28,NaN,3.0,33.0,37.0,NaN,20.0,16.0,4.0,NaN


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 8760 non-null   datetime64[ns]
 1   Hour  (GMT)          8760 non-null   object        
 2   Benceno (µg/m3)      8167 non-null   float64       
 3   CO (mg/m3)           8694 non-null   float64       
 4   CO 8h (mg/m3)        8709 non-null   float64       
 5   Etilbenceno (µg/m3)  8167 non-null   float64       
 6   NO (µg/m3)           8713 non-null   float64       
 7   NO2 (µg/m3)          8713 non-null   float64       
 8   NOX (µg/m3)          8713 non-null   float64       
 9   Ortoxileno (µg/m3)   8167 non-null   float64       
 10  PM10 (µg/m3)         8720 non-null   float64       
 11  PM2,5 (µg/m3)        8727 non-null   float64       
 12  SO2 (µg/m3)          8718 non-null   float64       
 13  Tolueno (µg/m3)      8167 non-nul

In [19]:
df.columns

Index(['Date', 'Hour  (GMT)', 'Benceno (µg/m3)', 'CO (mg/m3)', 'CO 8h (mg/m3)',
       'Etilbenceno (µg/m3)', 'NO (µg/m3)', 'NO2 (µg/m3)', 'NOX (µg/m3)',
       'Ortoxileno (µg/m3)', 'PM10 (µg/m3)', 'PM2,5 (µg/m3)', 'SO2 (µg/m3)',
       'Tolueno (µg/m3)'],
      dtype='object')

In [20]:
df= df.interpolate(method='linear', limit_direction='both')

C:\Users\ferna\AppData\Local\Temp\ipykernel_13948\3600762531.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df= df.interpolate(method='linear', limit_direction='both')


In [21]:
df['Hour  (GMT)']= df['Hour  (GMT)'].apply(lambda x: 'H'+str(x.split(':')[0]))

In [22]:
df.head()

,Date,Hour (GMT),Benceno (µg/m3),CO (mg/m3),CO 8h (mg/m3),Etilbenceno (µg/m3),NO (µg/m3),NO2 (µg/m3),NOX (µg/m3),Ortoxileno (µg/m3),PM10 (µg/m3),"PM2,5 (µg/m3)",SO2 (µg/m3),Tolueno (µg/m3)
0,2023-01-01,H01,0.09,0.19,0.17,0.04,3.0,12.0,16.0,0.40,33.0,12.0,5.0,0.77
1,2023-01-01,H02,0.09,0.19,0.17,0.04,2.0,8.0,12.0,0.40,31.0,12.0,4.0,0.77
2,2023-01-01,H03,0.09,0.21,0.18,0.04,1.0,7.0,9.0,0.40,31.0,12.0,4.0,0.77
3,2023-01-01,H04,0.09,0.22,0.19,0.06,1.0,12.0,14.0,0.44,33.0,13.0,4.0,0.79
4,2023-01-01,H05,0.06,0.23,0.19,0.00,2.0,20.0,22.0,0.41,31.0,13.0,4.0,0.74


In [23]:
df.columns

Index(['Date', 'Hour  (GMT)', 'Benceno (µg/m3)', 'CO (mg/m3)', 'CO 8h (mg/m3)',
       'Etilbenceno (µg/m3)', 'NO (µg/m3)', 'NO2 (µg/m3)', 'NOX (µg/m3)',
       'Ortoxileno (µg/m3)', 'PM10 (µg/m3)', 'PM2,5 (µg/m3)', 'SO2 (µg/m3)',
       'Tolueno (µg/m3)'],
      dtype='object')

In [24]:
df_pivoted= df.pivot(index="Date", columns='Hour  (GMT)')
df_pivoted.head()

Benceno (µg/m3)                                                \
Hour  (GMT)             H01       H02       H03       H04       H05   H06   
Date                                                                        
2023-01-01             0.09  0.090000  0.090000  0.090000  0.060000  0.13   
2023-01-02             0.08  0.076667  0.073333  0.070000  0.100000  0.10   
2023-01-03             0.39  0.310000  0.310000  0.310000  0.310000  0.33   
2023-01-04             0.48  0.410000  0.360000  0.356667  0.353333  0.35   
2023-01-05             0.59  0.490000  0.480000  0.480000  0.510000  0.54   

                                     ... Tolueno (µg/m3)                    \
Hour  (GMT)   H07   H08   H09   H10  ...             H15   H16   H17   H18   
Date                                 ...                                     
2023-01-01   0.08  0.08  0.08  0.06  ...            0.60  0.56  0.72  0.99   
2023-01-02   0.15  0.29  0.36  0.29  ...            1.30  0.82  0.77  1.18   
2023-01-03   0.35  0.38  0.42  0.47  ...            1.40  1.02  0.92  1.16   
2023-01-04   0.53  0.61  0.50  0.58  ...            3.85  4.11  4.69  4.98   
2023-01-05   0.57  0.69  0.74  0.68  ...            3.31  2.62  2.50  3.08   

                                                 
Hour  (GMT)   H19   H20   H21   H22   H23   H24  
Date                                             
2023-01-01   1.20  1.34  0.92  0.66  0.46  0.35  
2023-01-02   1.09  1.18  1.95  2.07  2.08  1.88  
2023-01-03   1.70  2.51  2.50  2.39  2.33  2.16  
2023-01-04   4.62  5.35  5.21  4.24  3.59  2.90  
2023-01-05   3.57  3.64  3.60  3.15  3.10  2.80  

[5 rows x 288 columns]

In [25]:
df_pivoted= df_pivoted.stack(level=0).reset_index()
df_pivoted
df_pivoted.head()

C:\Users\ferna\AppData\Local\Temp\ipykernel_13948\4171084373.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_pivoted= df_pivoted.stack(level=0).reset_index()


Hour (GMT),Date,level_1,H01,H02,H03,H04,H05,H06,H07,H08,...,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
0,2023-01-01,Benceno (µg/m3),0.09,0.09,0.09,0.09,0.06,0.13,0.08,0.08,...,0.02,0.02,0.00,0.05,0.07,0.11,0.12,0.07,0.08,0.04
1,2023-01-01,CO (mg/m3),0.19,0.19,0.21,0.22,0.23,0.23,0.22,0.21,...,0.17,0.19,0.22,0.22,0.21,0.23,0.29,0.29,0.37,0.35
2,2023-01-01,CO 8h (mg/m3),0.17,0.17,0.18,0.19,0.19,0.20,0.21,0.21,...,0.20,0.20,0.20,0.20,0.20,0.20,0.21,0.22,0.25,0.25
3,2023-01-01,Etilbenceno (µg/m3),0.04,0.04,0.04,0.06,0.00,0.03,0.05,0.06,...,0.01,0.00,0.00,0.03,0.01,0.09,0.03,0.00,0.00,0.00
4,2023-01-01,NO (µg/m3),3.00,2.00,1.00,1.00,2.00,2.00,2.00,2.00,...,2.00,2.00,2.00,3.00,3.00,2.00,2.00,2.00,2.00,1.00


In [26]:
df_pivoted['level_1']= df_pivoted['level_1'].apply(lambda x: bb_magnitude_dict[x])

In [27]:
df_pivoted['id']= a.rsplit('.',1)[0]

In [28]:
df_pivoted= df_pivoted.rename(columns={'Date': 'date', 'level_1':'pollutant'})

In [29]:
df_pivoted= df_pivoted.reset_index(drop=True)

In [30]:
df_pivoted

Hour (GMT),date,pollutant,H01,H02,H03,H04,H05,H06,H07,H08,...,H16,H17,H18,H19,H20,H21,H22,H23,H24,id
0,2023-01-01,BEN,0.09,0.09,0.09,0.09,0.06,0.13,0.08,0.08,...,0.02,0.00,0.05,0.07,0.11,0.12,0.07,0.08,0.04,M_DIAZ_HARO
1,2023-01-01,CO,0.19,0.19,0.21,0.22,0.23,0.23,0.22,0.21,...,0.19,0.22,0.22,0.21,0.23,0.29,0.29,0.37,0.35,M_DIAZ_HARO
2,2023-01-01,CO-8h,0.17,0.17,0.18,0.19,0.19,0.20,0.21,0.21,...,0.20,0.20,0.20,0.20,0.20,0.21,0.22,0.25,0.25,M_DIAZ_HARO
3,2023-01-01,EBE,0.04,0.04,0.04,0.06,0.00,0.03,0.05,0.06,...,0.00,0.00,0.03,0.01,0.09,0.03,0.00,0.00,0.00,M_DIAZ_HARO
4,2023-01-01,NO,3.00,2.00,1.00,1.00,2.00,2.00,2.00,2.00,...,2.00,2.00,3.00,3.00,2.00,2.00,2.00,2.00,1.00,M_DIAZ_HARO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4375,2023-12-31,OXY,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.01,0.10,0.23,0.29,0.48,0.58,0.62,0.62,0.62,M_DIAZ_HARO
4376,2023-12-31,PM10,5.00,6.00,7.00,11.00,7.00,5.00,3.00,4.00,...,13.00,12.00,11.00,13.00,25.00,33.00,16.00,20.00,86.00,M_DIAZ_HARO
4377,2023-12-31,PM25,2.00,3.00,4.00,6.00,4.00,3.00,2.00,2.00,...,6.00,5.00,6.00,8.00,19.00,28.00,12.00,16.00,16.00,M_DIAZ_HARO
4378,2023-12-31,SO2,2.00,2.00,2.00,2.00,4.00,5.00,3.00,3.00,...,6.00,7.00,8.00,7.00,7.00,6.00,4.00,4.00,20.00,M_DIAZ_HARO


In [31]:
df_pivoted= df_pivoted[(df_pivoted['date']>= bb_initdate) & (df_pivoted['date']<=bb_enddate)]
df_pivoted.head()

Hour (GMT),date,pollutant,H01,H02,H03,H04,H05,H06,H07,H08,...,H16,H17,H18,H19,H20,H21,H22,H23,H24,id
3120,2023-09-18,BEN,0.03,0.00,0.00,0.00,0.00,0.00,0.01,0.03,...,0.00,0.02,0.00,0.00,0.00,0.03,0.01,0.00,0.00,M_DIAZ_HARO
3121,2023-09-18,CO,0.20,0.20,0.22,0.22,0.22,0.23,0.23,0.23,...,0.21,0.20,0.23,0.24,0.27,0.27,0.27,0.25,0.25,M_DIAZ_HARO
3122,2023-09-18,CO-8h,0.27,0.27,0.26,0.26,0.24,0.23,0.22,0.22,...,0.22,0.22,0.22,0.22,0.23,0.23,0.24,0.24,0.25,M_DIAZ_HARO
3123,2023-09-18,EBE,0.14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,M_DIAZ_HARO
3124,2023-09-18,NO,0.00,0.00,0.00,0.00,1.00,1.00,3.00,3.00,...,1.00,1.00,1.00,1.00,1.00,1.00,0.00,0.00,0.00,M_DIAZ_HARO


In [ ]:
print("That's all folks!")